In [1]:
!pip install git+https://github.com/EmotionEngineer/actix.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, BatchNormalization, Activation, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import constraints
import numpy as np
import pandas as pd
import time
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_diabetes, fetch_california_housing

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')
warnings.filterwarnings('ignore', category=FutureWarning, module='tensorflow')

# --- 1. Import and Setup Activations from Actix ---
try:
    import actix
except ImportError:
    print("Error: 'actix' library is not installed.")
    print("Please install it using the command: pip install actix")
    sys.exit(1)

# Define the List of Activation Functions to Test ---
best_of_last_run = [
    'ParametricLogish', 
    'OptimA',
    'OptimXTemporal',
    'WeibullSoftplusActivation', 
    'A_ELuC',
    'ParametricBetaSoftsign'
]

# New functions to add to the test
new_to_test = [
    'A_STReLU', 
    'ATanSigU', 
    'SwishLogTanh', 
    'ArcGaLU', 
    'ParametricHyperbolicQuadraticActivation', 
    'RootSoftplus',
    'AdaptiveSinusoidalSoftgate', 
    'ExpTanhGatedActivation', 
    'HybridSinExpUnit',
    'ParametricLogarithmicSwish', 
    'AdaptiveCubicSigmoid', 
    'SmoothedAbsoluteGatedUnit',
    'GaussianTanhHarmonicUnit', 
    'SymmetricParametricRationalSigmoid', 
    'AdaptivePolynomialSwish',
    'LogSigmoidGatedElu', 
    'AdaptiveBipolarExponentialUnit', 
    'ParametricHyperGaussianGate',
    'TanhGatedArcsinhLinearUnit', 
    'ParametricOddPowerSwish', 
    'AdaptiveLinearLogTanh'
]

# Combine the lists for the current experiment
activations_to_test = best_of_last_run + new_to_test

# Dynamically create the activation map from the actix library
CUSTOM_ACTIVATIONS_MAP = {}
print("Loading activation functions from actix...")
for name in activations_to_test:
    try:
        # Get the activation class by its name from the actix module
        activation_class = getattr(actix, name)
        CUSTOM_ACTIVATIONS_MAP[name] = activation_class
        # print(f"  - {name}: Loaded successfully")
    except AttributeError:
        print(f"  - WARNING: Activation function '{name}' not found in the actix library and will be skipped.")
print("Loading complete.\n")

2025-06-14 08:34:23.620186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749890063.788499      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749890063.837930      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading activation functions from actix...
Loading complete.



In [3]:
# --- 2. Constants and Experiment Configuration ---
NUM_SEEDS = 5
EPOCHS = 150
PATIENCE = 40
BATCH_SIZE = 16
LEARNING_RATE = 1e-2

In [4]:
# --- 3. Layer Naming Utilities ---
_layer_name_counters = {}

def _get_unique_layer_name(base_name_key):
    if base_name_key not in _layer_name_counters:
        _layer_name_counters[base_name_key] = 0
    _layer_name_counters[base_name_key] += 1
    return f"{base_name_key}_{_layer_name_counters[base_name_key]}"

def _reset_layer_name_counters():
    global _layer_name_counters
    _layer_name_counters = {}

def _add_activation_layer(model, activation_name_str):
    unique_name = _get_unique_layer_name(activation_name_str.lower().replace(" ", "_").replace("-","_"))
    if activation_name_str in CUSTOM_ACTIVATIONS_MAP:
        # Use a custom activation from actix
        model.add(CUSTOM_ACTIVATIONS_MAP[activation_name_str](name=unique_name))
    else:
        # Use a standard TensorFlow activation
        model.add(Activation(activation_name_str, name=unique_name))

In [5]:
# --- 4. Model Definition for Regression Tasks ---
def create_regression_model(input_shape, activation_name_str, model_base_name="regression"):
    model_name_prefix = activation_name_str.replace(' ','_').replace("-","_").lower()
    model = Sequential(name=f"{model_base_name}_model_{model_name_prefix}")

    model.add(Input(shape=input_shape, name=_get_unique_layer_name("input_layer")))

    model.add(Dense(128, name=_get_unique_layer_name("dense")))
    _add_activation_layer(model, activation_name_str)
    model.add(BatchNormalization(name=_get_unique_layer_name("bn")))

    model.add(Dense(64, name=_get_unique_layer_name("dense")))
    _add_activation_layer(model, activation_name_str)
    model.add(BatchNormalization(name=_get_unique_layer_name("bn")))

    model.add(Dense(32, name=_get_unique_layer_name("dense")))
    _add_activation_layer(model, activation_name_str)
    model.add(BatchNormalization(name=_get_unique_layer_name("bn")))

    model.add(Dense(1, activation='linear', name=_get_unique_layer_name("output_dense")))

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error', # Primary metric for optimization
                  metrics=['mse', 'mae'])    # Additional metrics for reporting
    return model

In [6]:
# --- 5. Data Loading and Preprocessing ---
def load_and_preprocess_diabetes():
    diabetes = load_diabetes()
    X, y = diabetes.data, diabetes.target
    y = y.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler_X = StandardScaler()
    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)

    scaler_y = StandardScaler()
    y_train = scaler_y.fit_transform(y_train)
    y_test = scaler_y.transform(y_test)

    return (X_train, y_train), (X_test, y_test), scaler_y

def load_and_preprocess_california_housing():
    housing = fetch_california_housing()
    X, y = housing.data, housing.target
    y = y.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler_X = StandardScaler()
    X_train = scaler_X.fit_transform(X_train)
    X_test = scaler_X.transform(X_test)

    scaler_y = StandardScaler()
    y_train = scaler_y.fit_transform(y_train)
    y_test = scaler_y.transform(y_test)

    return (X_train, y_train), (X_test, y_test), scaler_y

In [7]:
# --- 6. Main Training and Evaluation Loop ---
if __name__ == '__main__':
    print(f"TensorFlow Version: {tf.__version__}")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Num GPUs Available: {len(gpus)}, Memory growth enabled.")
            print("TensorFlow is using GPU.")
        except RuntimeError as e:
            print(e)
    else:
        print("TensorFlow is using CPU.")

    datasets_to_benchmark = {
        "Diabetes": load_and_preprocess_diabetes,
        "CaliforniaHousing": load_and_preprocess_california_housing,
    }

    standard_activations = ['relu', 'gelu', 'swish', 'mish', 'sigmoid']
    # Test standard activations and all found in actix
    activations_to_test = standard_activations + list(CUSTOM_ACTIVATIONS_MAP.keys())
    
    all_results_list = []

    for dataset_name, load_fn in datasets_to_benchmark.items():
        print(f"\n\n--- Benchmarking on Dataset: {dataset_name} ---")
        (x_train, y_train), (x_test, y_test), y_scaler = load_fn()
        input_shape = (x_train.shape[1],)

        for act_name in activations_to_test:
            print(f"\n  --- Testing Activation: {act_name} on {dataset_name} ---")
            losses_for_activation = []
            mses_for_activation = []
            maes_for_activation = []
            
            # Pre-check model build
            try:
                tf.keras.backend.clear_session()
                _reset_layer_name_counters()
                _ = create_regression_model(input_shape, act_name, model_base_name=dataset_name.lower())
            except Exception as e:
                print(f"    ERROR: Could not build model with {act_name} for {dataset_name}. Error: {e}")
                result_entry = {
                    'dataset': dataset_name, 'activation': act_name,
                    'mean_loss': np.nan, 'std_loss': np.nan, 'best_loss': np.nan,
                    'mean_mse': np.nan, 'std_mse': np.nan, 'best_mse': np.nan,
                    'mean_mae': np.nan, 'std_mae': np.nan, 'best_mae': np.nan,
                    'notes': f'Failed to build model: {e}'
                }
                all_results_list.append(result_entry)
                continue

            for i in range(NUM_SEEDS):
                seed = 42 + i
                print(f"    Seed {i+1}/{NUM_SEEDS} (Actual seed: {seed}) for {act_name} on {dataset_name}")
                tf.keras.utils.set_random_seed(seed)
                tf.keras.backend.clear_session()
                _reset_layer_name_counters()

                model = create_regression_model(input_shape, act_name, model_base_name=dataset_name.lower())
                
                callbacks = [
                    EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, mode='min', verbose=0),
                ]
                
                start_time = time.time()
                try:
                    history = model.fit(x_train, y_train,
                                        batch_size=BATCH_SIZE,
                                        epochs=EPOCHS,
                                        validation_data=(x_test, y_test),
                                        callbacks=callbacks,
                                        verbose=0)
                    
                    # evaluate returns [loss, mse, mae] as per compiled metrics
                    eval_results = model.evaluate(x_test, y_test, verbose=0)
                    loss, mse, mae = eval_results[0], eval_results[1], eval_results[2]
                    
                    losses_for_activation.append(loss)
                    mses_for_activation.append(mse)
                    maes_for_activation.append(mae)
                    print(f"      Seed {i+1} Loss (MSE): {loss:.4f}, MAE: {mae:.4f}")

                except Exception as e:
                    print(f"      ERROR during training/evaluation for {act_name} with seed {seed} on {dataset_name}: {e}")
                    losses_for_activation.append(np.nan)
                    mses_for_activation.append(np.nan)
                    maes_for_activation.append(np.nan)
                
                end_time = time.time()
                # print(f"      Seed {i+1} training/eval time: {(end_time - start_time):.2f} seconds")

            # Calculate final statistics across all seeds
            if any(not np.isnan(l) for l in losses_for_activation):
                mean_loss = np.nanmean(losses_for_activation)
                std_loss = np.nanstd(losses_for_activation)
                best_loss = np.nanmin(losses_for_activation)

                mean_mse = np.nanmean(mses_for_activation)
                std_mse = np.nanstd(mses_for_activation)
                best_mse = np.nanmin(mses_for_activation)

                mean_mae = np.nanmean(maes_for_activation)
                std_mae = np.nanstd(maes_for_activation)
                best_mae = np.nanmin(maes_for_activation)

                result_entry = {
                    'dataset': dataset_name, 'activation': act_name,
                    'mean_loss': mean_loss, 'std_loss': std_loss, 'best_loss': best_loss,
                    'mean_mse': mean_mse, 'std_mse': std_mse, 'best_mse': best_mse,
                    'mean_mae': mean_mae, 'std_mae': std_mae, 'best_mae': best_mae,
                    'notes': ''
                }
                print(f"    Results for {act_name} on {dataset_name}: Mean Loss: {mean_loss:.4f}, Mean MAE: {mean_mae:.4f}")
            else:
                result_entry = {
                    'dataset': dataset_name, 'activation': act_name,
                    'mean_loss': np.nan, 'std_loss': np.nan, 'best_loss': np.nan,
                    'mean_mse': np.nan, 'std_mse': np.nan, 'best_mse': np.nan,
                    'mean_mae': np.nan, 'std_mae': np.nan, 'best_mae': np.nan,
                    'notes': 'All runs failed or build failed'
                }
            all_results_list.append(result_entry)

    # --- 7. Output and Saving Results ---
    print("\n\n--- Final Comparison Results ---")
    results_df = pd.DataFrame(all_results_list)
    
    # Sort by dataset, then by mean_loss (MSE)
    results_df = results_df.sort_values(by=['dataset', 'mean_loss'], ascending=[True, True])
    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)

    print(results_df[['dataset', 'activation', 'mean_loss', 'std_loss', 'best_loss', 'mean_mae', 'std_mae', 'best_mae', 'notes']])

    # Save to CSV
    output_filename = "regression_activation_comparison_results.csv"
    results_df.to_csv(output_filename, index=False)
    print(f"\nResults saved to {output_filename}")

TensorFlow Version: 2.18.0
Num GPUs Available: 1, Memory growth enabled.
TensorFlow is using GPU.


--- Benchmarking on Dataset: Diabetes ---

  --- Testing Activation: relu on Diabetes ---


I0000 00:00:1749890081.164893      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


    Seed 1/5 (Actual seed: 42) for relu on Diabetes


I0000 00:00:1749890085.783141      94 service.cc:148] XLA service 0x7d9bac0101a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749890085.783641      94 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1749890086.123613      94 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749890087.096070      94 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


      Seed 1 Loss (MSE): 0.4542, MAE: 0.5549
    Seed 2/5 (Actual seed: 43) for relu on Diabetes
      Seed 2 Loss (MSE): 0.5317, MAE: 0.5996
    Seed 3/5 (Actual seed: 44) for relu on Diabetes
      Seed 3 Loss (MSE): 0.4424, MAE: 0.5335
    Seed 4/5 (Actual seed: 45) for relu on Diabetes
      Seed 4 Loss (MSE): 0.5335, MAE: 0.6281
    Seed 5/5 (Actual seed: 46) for relu on Diabetes
      Seed 5 Loss (MSE): 0.5013, MAE: 0.5976
    Results for relu on Diabetes: Mean Loss: 0.4926, Mean MAE: 0.5827

  --- Testing Activation: gelu on Diabetes ---
    Seed 1/5 (Actual seed: 42) for gelu on Diabetes
      Seed 1 Loss (MSE): 0.4655, MAE: 0.5399
    Seed 2/5 (Actual seed: 43) for gelu on Diabetes
      Seed 2 Loss (MSE): 0.4789, MAE: 0.5529
    Seed 3/5 (Actual seed: 44) for gelu on Diabetes
      Seed 3 Loss (MSE): 0.4048, MAE: 0.4942
    Seed 4/5 (Actual seed: 45) for gelu on Diabetes
      Seed 4 Loss (MSE): 0.5397, MAE: 0.6245
    Seed 5/5 (Actual seed: 46) for gelu on Diabetes
      See

/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1741: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)


    Seed 1/5 (Actual seed: 42) for SmoothedAbsoluteGatedUnit on CaliforniaHousing
      Seed 1 Loss (MSE): 0.3372, MAE: 0.4496
    Seed 2/5 (Actual seed: 43) for SmoothedAbsoluteGatedUnit on CaliforniaHousing
      Seed 2 Loss (MSE): 0.3328, MAE: 0.4268
    Seed 3/5 (Actual seed: 44) for SmoothedAbsoluteGatedUnit on CaliforniaHousing
      Seed 3 Loss (MSE): 0.2635, MAE: 0.3509
    Seed 4/5 (Actual seed: 45) for SmoothedAbsoluteGatedUnit on CaliforniaHousing
      Seed 4 Loss (MSE): 0.2853, MAE: 0.3625
    Seed 5/5 (Actual seed: 46) for SmoothedAbsoluteGatedUnit on CaliforniaHousing
      Seed 5 Loss (MSE): 0.2416, MAE: 0.3374
    Results for SmoothedAbsoluteGatedUnit on CaliforniaHousing: Mean Loss: 0.2921, Mean MAE: 0.3854

  --- Testing Activation: GaussianTanhHarmonicUnit on CaliforniaHousing ---
    Seed 1/5 (Actual seed: 42) for GaussianTanhHarmonicUnit on CaliforniaHousing
      Seed 1 Loss (MSE): 0.2253, MAE: 0.3305
    Seed 2/5 (Actual seed: 43) for GaussianTanhHarmonicUnit on

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
